In [1]:
!nvidia-smi

Sat Nov 23 12:11:26 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.51.01              Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8              1W /   60W |       0MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
SCRIPT_DIR = os.path.abspath("")

In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key=os.environ['ROBOFLOW_TOKEN'])
project = rf.workspace("sushiteam").project("litter-dataset-fxyqc")
version = project.version(2)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Litter-Dataset-2 in yolov11:: 100


In [2]:
from dataset_util import merge

train_sets = [os.path.join(SCRIPT_DIR, 'TACO/data/train'), 
              os.path.join(SCRIPT_DIR, 'TACO_TN_UAV/train'), 
              os.path.join(SCRIPT_DIR, 'coco_human/data/train'),
              os.path.join(SCRIPT_DIR, 'people_detection/data/train')]
taco_index = [0,1]

merge.merge_yolo(train_sets, os.path.join(SCRIPT_DIR, 'merged_data/train'), taco_index)

output_dir = os.path.join(SCRIPT_DIR, 'merged_data')
yaml_content = """train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['Person', 'Trash']
"""

with open(f"{output_dir}/data.yaml", "w") as yaml_file:
    yaml_file.write(yaml_content)

Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO/data/train completed, Counter: 0
Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO_TN_UAV/train completed, Counter: 1
Dataset /mnt/c/Adrianov/Projects/trash-detector/coco_human/data/train completed, Counter: 2
Dataset /mnt/c/Adrianov/Projects/trash-detector/people_detection/data/train completed, Counter: 3
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/train


In [4]:
validation_sets = [os.path.join(SCRIPT_DIR, 'TACO/data/valid'), 
              os.path.join(SCRIPT_DIR, 'TACO_TN_UAV/valid'), 
              os.path.join(SCRIPT_DIR, 'coco_human/data/valid'),
              os.path.join(SCRIPT_DIR, 'people_detection/data/valid')]
taco_index = [0,1]

merge.merge_yolo(validation_sets, os.path.join(SCRIPT_DIR, 'merged_data/valid'), taco_index)


Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO/data/valid completed, Counter: 0
Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO_TN_UAV/valid completed, Counter: 1
Dataset /mnt/c/Adrianov/Projects/trash-detector/coco_human/data/valid completed, Counter: 2
Dataset /mnt/c/Adrianov/Projects/trash-detector/people_detection/data/valid completed, Counter: 3
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/valid


In [6]:
test_sets = [os.path.join(SCRIPT_DIR, 'TACO/data/test'), 
              os.path.join(SCRIPT_DIR, 'TACO_TN_UAV/test'), 
              os.path.join(SCRIPT_DIR, 'coco_human/data/test'),
              os.path.join(SCRIPT_DIR, 'people_detection/data/test')]
taco_index = [0,1]

merge.merge_yolo(test_sets, os.path.join(SCRIPT_DIR, 'merged_data/test'), taco_index)


Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO/data/test completed, Counter: 0
Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO_TN_UAV/test completed, Counter: 1
Dataset /mnt/c/Adrianov/Projects/trash-detector/coco_human/data/test completed, Counter: 2
Dataset /mnt/c/Adrianov/Projects/trash-detector/people_detection/data/test completed, Counter: 3
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/test


In [2]:
from dataset_util import merge

train_sets = [os.path.join(SCRIPT_DIR, 'Litter-Dataset-2/train')]
taco_index = [0]
seg_index = [0]

merge.merge_yolo(train_sets, os.path.join(SCRIPT_DIR, 'merged_data/train'), taco_index, start_idx=14863, seg_idx=seg_index)

output_dir = os.path.join(SCRIPT_DIR, 'merged_data')

Dataset /mnt/c/Adrianov/Projects/trash-detector/Litter-Dataset-2/train completed, Counter: 0
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/train


In [3]:
valid_sets = [os.path.join(SCRIPT_DIR, 'Litter-Dataset-2/valid')]
taco_index = [0]
seg_index = [0]

merge.merge_yolo(valid_sets, os.path.join(SCRIPT_DIR, 'merged_data/valid'), taco_index, start_idx=2971, seg_idx=seg_index)

output_dir = os.path.join(SCRIPT_DIR, 'merged_data')

Dataset /mnt/c/Adrianov/Projects/trash-detector/Litter-Dataset-2/valid completed, Counter: 0
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/valid


In [4]:
test_sets = [os.path.join(SCRIPT_DIR, 'Litter-Dataset-2/test')]
taco_index = [0]
seg_index = [0]

merge.merge_yolo(test_sets, os.path.join(SCRIPT_DIR, 'merged_data/test'), taco_index, start_idx=1742, seg_idx=seg_index)

output_dir = os.path.join(SCRIPT_DIR, 'merged_data')

Dataset /mnt/c/Adrianov/Projects/trash-detector/Litter-Dataset-2/test completed, Counter: 0
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/test


In [ ]:
from ultralytics import YOLO

model = YOLO(os.path.join(SCRIPT_DIR, 'model/TrashDetectorv2/weights/last.pt')) 
merged_data = os.path.join(SCRIPT_DIR, 'merged_data/data.yaml')
output_dir = os.path.join(SCRIPT_DIR, 'model')
results = model.train(name='TrashDetectorv2', data=merged_data, epochs=100, 
                      imgsz=416, freeze=11, batch=32, device=0, project=output_dir, 
                      resume=True, plots=True, exist_ok=True,
                      save_period=10)

New https://pypi.org/project/ultralytics/8.3.39 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.29 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine/trainer: task=detect, mode=train, model=/mnt/c/Adrianov/Projects/trash-detector/model/TrashDetectorv2/weights/last.pt, data=/mnt/c/Adrianov/Projects/trash-detector/merged_data/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=416, save=True, save_period=10, cache=False, device=0, workers=8, project=/mnt/c/Adrianov/Projects/trash-detector/model, name=TrashDetectorv2, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=/mnt/c/Adrianov/Projects/trash-detector/model/TrashDetectorv2/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=11, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=F

train: Scanning /mnt/c/Adrianov/Projects/trash-detector/merged_data


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /mnt/c/Adrianov/Projects/trash-detector/merged_data/v

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 61, len(boxes) = 8837. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

     89/100      2.38G      1.176      1.046       1.12         84 
                 Class     Images  Instances      Box(P          R 


                   all       3218       8837      0.752      0.524      0.601      0.387

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     90/100      2.39G      1.176      1.041      1.119         95 
                 Class     Images  Instances      Box(P          R 


                   all       3218       8837      0.749      0.525      0.601      0.387
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     91/100      2.88G      1.158     0.9737      1.078         58 
                 Class     Images  Instances      Box(P          R 